In [1]:
# Prepare

%matplotlib notebook

import numpy as np
import pandas as pd
import scipy
import time
import warnings

from sklearn.linear_model import LinearRegression

In [2]:
efeatures_NMA=pd.read_csv('../data/efeatures_NMA.csv')
Xe=np.array(efeatures_NMA.drop(columns=['data','ttype','cellnames','gmm_cluster']))

geneExp_NMA=pd.read_csv('../data/geneExp_NMA.csv')
Xg=np.array(geneExp_NMA.drop(columns=['data','ttype','cellnames','gmm_cluster']))

Xe = Xe - np.mean(Xe, axis=0)
Xe = Xe / np.std(Xe, axis=0)

Xg = Xg - np.mean(Xg, axis=0)
Xg = Xg / np.std(Xg, axis=0)

print('Shape of Xe:', Xe.shape, '\nShape of Xg:', Xg.shape)

Shape of Xe: (3654, 20) 
Shape of Xg: (3654, 20)


In [3]:
Xg_3=Xg[:,0:3:]
Xe_3=Xe[:,0:3:]

reg = LinearRegression().fit(Xg_3,Xe_3)
r2 = reg.score(Xg_3, Xe_3, sample_weight=None)

r2

0.9871211142869584

## Cross Validation

In [4]:
def cv(X, Y, reps=20, folds=10, dims=np.array([3,5,7,9,11,13,15,20]), seed=42):

    r2 = np.zeros((folds, reps, len(dims))) * np.nan
    
    np.random.seed(seed)
    t = time.time()
    n=X.shape[0]
    for m,dim in enumerate(dims):
            X = Xg[:,0:dim:]
            Y = Xe[:,0:dim:]

            for rep in range(reps):
                #print(rep+1, end='')
                ind = np.random.permutation(n)
                X = X[ind,:]
                Y = Y[ind,:]
            
                # CV folds
                for cvfold in range(folds):
                    #print('.', end='')
    
                    indtest  = np.arange(cvfold*int(n/folds), (cvfold+1)*int(n/folds))
                    indtrain = np.setdiff1d(np.arange(n), indtest)
                    Xtrain = np.copy(X[indtrain,:])
                    Ytrain = np.copy(Y[indtrain,:])
                    Xtest  = np.copy(X[indtest,:])
                    Ytest  = np.copy(Y[indtest,:])
                
                    # mean centering
                    X_mean = np.mean(Xtrain, axis=0)
                    Xtrain -= X_mean
                    Xtest  -= X_mean
                    Y_mean = np.mean(Ytrain, axis=0)
                    Ytrain -= Y_mean
                    Ytest  -= Y_mean
                    
                    fit = LinearRegression().fit(Xtrain,Ytrain)
                    r2[cvfold, rep, m] =1-np.sum((fit.predict(Xtest)-Ytest)**2)/np.sum(Xtest**2)
                
    t = time.time() - t
    min,s = divmod(t, 60)
    h,min = divmod(min, 60)
    print('Time: {}h {:2.0f}min {:2.0f}s'.format(h,min,s))    
    
    return r2

In [5]:
cv_result = cv(X=Xg,Y=Xe)
cv_mean = np.nanmean(cv_result, axis=(0,1))

Time: 0.0h  0min  2s


In [6]:
cv_result

array([[[0.98698341, 0.97759188, 0.97196616, ..., 0.9527768 ,
         0.95897038, 0.95800144],
        [0.98759255, 0.97902416, 0.96930602, ..., 0.95534058,
         0.95100109, 0.9514697 ],
        [0.98767093, 0.975851  , 0.97018217, ..., 0.96097376,
         0.95350053, 0.9587538 ],
        ...,
        [0.98677081, 0.97722244, 0.97212889, ..., 0.9596977 ,
         0.96082043, 0.95898392],
        [0.98640727, 0.97510547, 0.97418837, ..., 0.95862859,
         0.96221041, 0.95211044],
        [0.98712465, 0.97819594, 0.97335368, ..., 0.96291588,
         0.95620995, 0.95736992]],

       [[0.98627679, 0.97948017, 0.96621637, ..., 0.9649463 ,
         0.96092966, 0.9467406 ],
        [0.98251099, 0.98127636, 0.97017558, ..., 0.9668448 ,
         0.96438154, 0.95685455],
        [0.98473644, 0.97477163, 0.97316392, ..., 0.95724298,
         0.96140691, 0.94850869],
        ...,
        [0.98679795, 0.98162414, 0.96880132, ..., 0.96024715,
         0.94510844, 0.95226623],
        [0.9

In [7]:
cv_mean

array([0.98708212, 0.97786596, 0.97098485, 0.96676881, 0.96235767,
       0.95930368, 0.9573597 , 0.95410385])